In [1]:
import imageio
import numpy as np
import json
from matplotlib import pyplot as plt
from skimage.transform import warp
from scipy.ndimage import gaussian_filter
from scipy import ndimage
import math
import nibabel as nib
import utils
import random
import os

In [2]:
base_dir = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED'

width = 224
height = 224

n_x = 5
n_y = 5

sample_size = 4800

p_dropout = 0.7
p_sinus = 0.7
p_rot = 0.7
p_scale = 0.7
p_shear = 0.7
p_global_trans = 0.7
p_local_trans = 0.7

# -----------
dropout_max = 0.2

x_sin_max = 0.1 * height
y_sin_max = 0.05 * width

rot_max = math.pi * 0.1125 # 22.5 deg

x_scale_max = 1.25
y_scale_max = 1.25

x_shear_max = math.pi * 0.05625 # 11.25 deg
y_shear_max = math.pi * 0.05625 # 11.25 deg

x_global_max = 0.3 * width
y_global_max = 0.3 * height

x_local_max = 0.01 * width
y_local_max = 0.01 * height

In [3]:
spacing_x = (width / 2.0) / n_x
spacing_y = (height / 2.0) / n_y

base_offset_x = width / 4.0
base_offset_y = height / 4.0

offset_x = spacing_x / 2.0 + base_offset_x
offset_y = spacing_y / 2.0 + base_offset_y

In [4]:
def random_decision(probability):
    return random.random() < probability

In [5]:
def apply_sinus(x, y, rand_1, rand_2): 
    y_ratio = (y - offset_y) / (spacing_y * (n_y - 1))
    y_sin = math.sin(y_ratio * math.pi) * y_sin_max * rand_1
    u = x - round(y_sin)
    
    x_ratio = (x - offset_x) / (spacing_x * (n_x - 1))
    x_sin = math.sin(x_ratio * math.pi) * x_sin_max * rand_2
    v = y - round(x_sin)
    
    return u,v

def apply_rot(x, y, rand_1):
    shift_x = round(width / 2)
    shift_y = round(height / 2)
    
    x = x - shift_x
    y = y - shift_y
    
    alpha = rot_max * rand_1
    
    vec = np.array([x, y])
    mat = np.array([[np.cos(alpha), -np.sin(alpha)],
                    [np.sin(alpha), np.cos(alpha)]])

    vec = np.dot(mat, vec)
    
    u = int(round(vec[0]))
    v = int(round(vec[1]))
    
    u = u + shift_x
    v = v + shift_y
    
    return u,v

def apply_scale(x, y, rand_1, rand_2):
    rand_1 = (rand_1 + 1.0) / 2.0
    rand_2 = (rand_1 + 1.0) / 2.0
    
    shift_x = round(width / 2)
    shift_y = round(height / 2)
    
    x = x - shift_x
    y = y - shift_y
    
    x_scale = (1 / x_scale_max) + (x_scale_max - (1 / x_scale_max)) * rand_1
    y_scale = (1 / y_scale_max) + (y_scale_max - (1 / y_scale_max)) * rand_2
    
    vec = np.array([x, y])
    mat = np.array([[x_scale, 0      ],
                    [0      , y_scale]])

    vec = np.dot(mat, vec)
    
    u = int(round(vec[0]))
    v = int(round(vec[1]))
    
    u = u + shift_x
    v = v + shift_y
    
    return u,v

def apply_shear(x, y, rand_1, rand_2):
    shift_x = round(width / 2)
    shift_y = round(height / 2)
    
    x = x - shift_x
    y = y - shift_y
    
    phi = x_shear_max * rand_1
    psi = y_shear_max * rand_2
    
    vec = np.array([x, y])
    mat = np.array([[1          , np.tan(phi)],
                    [np.tan(psi), 1          ]])

    vec = np.dot(mat, vec)
    
    u = int(round(vec[0]))
    v = int(round(vec[1]))
    
    u = u + shift_x
    v = v + shift_y
    
    return u,v

def apply_global_trans(x, y, rand_1, rand_2):
    u = x + round(x_global_max * rand_1)
    v = y + round(y_global_max * rand_2)
    
    return u,v

def apply_local_trans(x, y):
    u = x + round(x_local_max * (random.random() * 2.0 - 1.0))
    v = y + round(y_local_max * (random.random() * 2.0 - 1.0))
    
    return u,v

def apply_dropout(rand):
    dropout = dropout_max * rand
    if random_decision(dropout):
        return True
        
    else:
        return False

In [6]:
def get_fixed_data(sample_index):
    is_dropout = random_decision(p_dropout)
    
    fixed = np.zeros((height, width))
    index_2_xy = dict()
    index = 0
    rand = random.random()
    for index_y in reversed(range(n_y)):
        y = int(round(offset_y + (spacing_y * index_y)))

        for index_x in range(n_x):
            x = int(round(offset_x + (spacing_x * index_x)))
            
            drop = apply_dropout(rand) if (is_dropout and sample_index!=0) else False
            
            if not drop:
                fixed[y][x] = 1
                index_2_xy[index] = (x, y)
            
            index += 1
            
    return fixed, index_2_xy

In [7]:
def get_moving_data(rand_1, rand_2, sample_index, index_2_xy):
    is_sinus = random_decision(p_sinus)
    is_rot = random_decision(p_rot)
    is_scale = random_decision(p_scale)
    is_shear = random_decision(p_shear)
    is_global_trans = random_decision(p_global_trans)
    is_local_trans = random_decision(p_local_trans)
    
    moving = np.zeros((height, width))
    index_2_uv = dict()
    index = 0
    for index_y in reversed(range(n_y)):
        y = int(round(offset_y + (spacing_y * index_y)))

        for index_x in range(n_x):
            if index in index_2_xy:
                x = int(round(offset_x + (spacing_x * index_x)))

                u = x
                v = y
                
                if sample_index != 0:
                    if is_sinus: u,v = apply_sinus(u, v, rand_1, rand_2)
                    if is_rot: u,v = apply_rot(u, v, rand_1) 
                    if is_scale: u,v = apply_scale(u, v, rand_1, rand_2) 
                    if is_shear: u,v = apply_shear(u, v, rand_1, rand_2) 
                    if is_global_trans: u,v = apply_global_trans(u, v, rand_1, rand_2) 
                    if is_local_trans: u,v = apply_local_trans(u, v) 
                
                if u < 0 or v < 0 or u > (width-1) or v > (height-1):
                    print('Additional attempt for sample index "{}"'.format(sample_index))
                    return None, None, False
                
                else:
                    moving[v][u] = 1
                    index_2_uv[index] = (u, v)

            index += 1
            
    return moving, index_2_uv, True

In [8]:
for sample_index in range(0, sample_size):
    # ----- fixed -----
    fixed, index_2_xy = get_fixed_data(sample_index)
    
    if sample_index == 0:
        json_file = json.dumps(index_2_xy)
        f = open(base_dir + "/raw/{}_f.json".format(sample_index), "w")
        f.write(json_file)
        f.close()

        fixed = utils.apply_smoothing(fixed)
        plt.imsave(base_dir + "/raw/{}_f.png".format(sample_index), fixed, cmap="gray")

    # ----- moving -------
    is_valid = False
    
    while(not is_valid):
        rand_1 = random.random() * 2.0 - 1.0
        rand_2 = random.random() * 2.0 - 1.0
        
        moving, index_2_uv, is_valid = get_moving_data(rand_1, rand_2, sample_index, index_2_xy)
        
    json_file = json.dumps(index_2_uv)
    f = open(base_dir + "/raw/{}_m.json".format(sample_index), "w")
    f.write(json_file)
    f.close()

    moving = utils.apply_smoothing(moving)
    plt.imsave(base_dir + "/raw/{}_m.png".format(sample_index), moving, cmap="gray")

Additional attempt for sample index "3"
Additional attempt for sample index "4"
Additional attempt for sample index "5"
Additional attempt for sample index "5"
Additional attempt for sample index "15"
Additional attempt for sample index "16"
Additional attempt for sample index "19"
Additional attempt for sample index "25"
Additional attempt for sample index "28"
Additional attempt for sample index "45"
Additional attempt for sample index "48"
Additional attempt for sample index "48"
Additional attempt for sample index "49"
Additional attempt for sample index "55"
Additional attempt for sample index "75"
Additional attempt for sample index "75"
Additional attempt for sample index "78"
Additional attempt for sample index "79"
Additional attempt for sample index "83"
Additional attempt for sample index "83"
Additional attempt for sample index "101"
Additional attempt for sample index "103"
Additional attempt for sample index "108"
Additional attempt for sample index "110"
Additional attem

Additional attempt for sample index "898"
Additional attempt for sample index "905"
Additional attempt for sample index "915"
Additional attempt for sample index "922"
Additional attempt for sample index "927"
Additional attempt for sample index "928"
Additional attempt for sample index "928"
Additional attempt for sample index "938"
Additional attempt for sample index "944"
Additional attempt for sample index "950"
Additional attempt for sample index "955"
Additional attempt for sample index "955"
Additional attempt for sample index "955"
Additional attempt for sample index "959"
Additional attempt for sample index "964"
Additional attempt for sample index "966"
Additional attempt for sample index "967"
Additional attempt for sample index "971"
Additional attempt for sample index "974"
Additional attempt for sample index "975"
Additional attempt for sample index "978"
Additional attempt for sample index "984"
Additional attempt for sample index "989"
Additional attempt for sample inde

Additional attempt for sample index "1792"
Additional attempt for sample index "1792"
Additional attempt for sample index "1794"
Additional attempt for sample index "1816"
Additional attempt for sample index "1825"
Additional attempt for sample index "1825"
Additional attempt for sample index "1827"
Additional attempt for sample index "1831"
Additional attempt for sample index "1832"
Additional attempt for sample index "1834"
Additional attempt for sample index "1837"
Additional attempt for sample index "1851"
Additional attempt for sample index "1857"
Additional attempt for sample index "1860"
Additional attempt for sample index "1867"
Additional attempt for sample index "1868"
Additional attempt for sample index "1872"
Additional attempt for sample index "1874"
Additional attempt for sample index "1882"
Additional attempt for sample index "1883"
Additional attempt for sample index "1897"
Additional attempt for sample index "1897"
Additional attempt for sample index "1911"
Additional 

Additional attempt for sample index "2674"
Additional attempt for sample index "2675"
Additional attempt for sample index "2678"
Additional attempt for sample index "2678"
Additional attempt for sample index "2686"
Additional attempt for sample index "2688"
Additional attempt for sample index "2688"
Additional attempt for sample index "2701"
Additional attempt for sample index "2728"
Additional attempt for sample index "2730"
Additional attempt for sample index "2738"
Additional attempt for sample index "2742"
Additional attempt for sample index "2746"
Additional attempt for sample index "2751"
Additional attempt for sample index "2753"
Additional attempt for sample index "2756"
Additional attempt for sample index "2758"
Additional attempt for sample index "2759"
Additional attempt for sample index "2767"
Additional attempt for sample index "2784"
Additional attempt for sample index "2791"
Additional attempt for sample index "2799"
Additional attempt for sample index "2801"
Additional 

Additional attempt for sample index "3570"
Additional attempt for sample index "3579"
Additional attempt for sample index "3582"
Additional attempt for sample index "3588"
Additional attempt for sample index "3588"
Additional attempt for sample index "3589"
Additional attempt for sample index "3595"
Additional attempt for sample index "3596"
Additional attempt for sample index "3601"
Additional attempt for sample index "3612"
Additional attempt for sample index "3627"
Additional attempt for sample index "3667"
Additional attempt for sample index "3667"
Additional attempt for sample index "3668"
Additional attempt for sample index "3668"
Additional attempt for sample index "3670"
Additional attempt for sample index "3672"
Additional attempt for sample index "3673"
Additional attempt for sample index "3677"
Additional attempt for sample index "3684"
Additional attempt for sample index "3685"
Additional attempt for sample index "3687"
Additional attempt for sample index "3688"
Additional 

Additional attempt for sample index "4535"
Additional attempt for sample index "4541"
Additional attempt for sample index "4543"
Additional attempt for sample index "4550"
Additional attempt for sample index "4552"
Additional attempt for sample index "4553"
Additional attempt for sample index "4565"
Additional attempt for sample index "4571"
Additional attempt for sample index "4578"
Additional attempt for sample index "4580"
Additional attempt for sample index "4582"
Additional attempt for sample index "4592"
Additional attempt for sample index "4593"
Additional attempt for sample index "4604"
Additional attempt for sample index "4609"
Additional attempt for sample index "4613"
Additional attempt for sample index "4613"
Additional attempt for sample index "4621"
Additional attempt for sample index "4624"
Additional attempt for sample index "4624"
Additional attempt for sample index "4625"
Additional attempt for sample index "4630"
Additional attempt for sample index "4645"
Additional 

In [9]:
# Image 0 will always be in the training
shuffled_list = list(range(1,sample_size))
random.shuffle(shuffled_list)
shuffled_list.insert(0,0)

if sample_size*0.15 % 1 > 0.0:
    raise AssertionError("Sample needs different split")

train_part = int(sample_size * 0.7)
val_part = int(sample_size * 0.15)
test_part = int(sample_size * 0.15)

train = shuffled_list[0:train_part] # 70 percent training
validation = shuffled_list[train_part:train_part+val_part] # 15 percent training
test = shuffled_list[train_part+val_part:sample_size] # 15 percent training

In [10]:
fixed_sou = os.path.join(base_dir, "raw", "0_f.png")
fixed_json_sou = os.path.join(base_dir, "raw", "0_f.json")

fixed_des = os.path.join(base_dir, "train", "0_f.png")
fixed_json_des = os.path.join(base_dir, "train", "0_f.json")

os.rename(fixed_sou, fixed_des)
os.rename(fixed_json_sou, fixed_json_des)

for index in train:
    moving_sou = os.path.join(base_dir, "raw", "{}_m.png".format(index))
    moving_json_sou = os.path.join(base_dir, "raw", "{}_m.json".format(index))
    
    moving_des = os.path.join(base_dir, "train", "{}_m.png".format(index))
    moving_json_des = os.path.join(base_dir, "train", "{}_m.json".format(index))
    
    os.rename(moving_sou, moving_des)
    os.rename(moving_json_sou, moving_json_des)

In [11]:
for index in validation:
    moving_sou = os.path.join(base_dir, "raw", "{}_m.png".format(index))
    moving_json_sou = os.path.join(base_dir, "raw", "{}_m.json".format(index))
    
    moving_des = os.path.join(base_dir, "validation", "{}_m.png".format(index))
    moving_json_des = os.path.join(base_dir, "validation", "{}_m.json".format(index))
    
    os.rename(moving_sou, moving_des)
    os.rename(moving_json_sou, moving_json_des)

In [12]:
for index in test:
    moving_sou = os.path.join(base_dir, "raw", "{}_m.png".format(index))
    moving_json_sou = os.path.join(base_dir, "raw", "{}_m.json".format(index))
    
    moving_des = os.path.join(base_dir, "test", "{}_m.png".format(index))
    moving_json_des = os.path.join(base_dir, "test", "{}_m.json".format(index))
    
    os.rename(moving_sou, moving_des)
    os.rename(moving_json_sou, moving_json_des)